In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
import pandas as pd
import os
from models import columns,vectorize_data
from resultsAnalyse import drawConfusionMatrix
import torch
from sklearn.metrics import f1_score
def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = load_dataset("higopires/RePro-categories-multilabel")
print(len(ds["train"]))
#remove INADEQUADA examples
ds = ds.filter(filter_inadequada)
print(len(ds["train"]))

8002
7674


In [2]:


print(ds["train"].features)
#run_blitr(train_df,val_df,test_df)


{'review_text': Value(dtype='string', id=None), 'ENTREGA': Value(dtype='int64', id=None), 'OUTROS': Value(dtype='int64', id=None), 'PRODUTO': Value(dtype='int64', id=None), 'CONDICOESDERECEBIMENTO': Value(dtype='int64', id=None), 'INADEQUADA': Value(dtype='int64', id=None), 'ANUNCIO': Value(dtype='int64', id=None)}


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to probabilities and then to binary predictions
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    
    return {'f1': float(f1)}

def preprocess_function(sample):
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Tokenize text
    tokenized = tokenizer(sample["review_text"], truncation=True, padding=True)
    
    # Convert binary label columns to a list (e.g., [1, 0, 1, 0, 0, 0])
    labels = []
    for i in range(len(sample["review_text"])):
        label_row = [
            float(sample["ENTREGA"][i]),
            float(sample["OUTROS"][i]),
            float(sample["PRODUTO"][i]),
            float(sample["CONDICOESDERECEBIMENTO"][i]),
            float(sample["ANUNCIO"][i])
        ]
        labels.append(label_row)
    
    tokenized["labels"] = labels
    return tokenized

model_name = "Qwen/Qwen3-8B"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5,problem_type="multi_label_classification", ignore_mismatched_sizes=True )

tokenized_dataset = ds.map(preprocess_function, batched=True)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from sklearn.metrics import f1_score
small_pred = np.array([[0, 1, 0], [1, 0, 1]])
small_true = np.array([[0, 1, 0], [1, 0, 0]])
print(f1_score(small_true, small_pred, average='samples'))

0.8333333333333333


In [5]:
print(tokenized_dataset["train"]["labels"][2])

[1.0, 0.0, 1.0, 0.0, 0.0]


In [6]:
training_args = TrainingArguments(
    output_dir="./resultsTransformer",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

#trainer.evaluate()

In [7]:
trainer.train()

  0%|          | 0/24000 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

{'eval_loss': 0.30067911744117737, 'eval_f1': 0.8002847988608046, 'eval_runtime': 6.2824, 'eval_samples_per_second': 151.533, 'eval_steps_per_second': 9.55, 'epoch': 1.0}
{'loss': 0.3943, 'grad_norm': 2.059049367904663, 'learning_rate': 9.791666666666666e-06, 'epoch': 1.04}


  0%|          | 0/60 [00:00<?, ?it/s]

{'eval_loss': 0.23694944381713867, 'eval_f1': 0.8448821667252903, 'eval_runtime': 5.7273, 'eval_samples_per_second': 166.221, 'eval_steps_per_second': 10.476, 'epoch': 2.0}
{'loss': 0.266, 'grad_norm': 2.1203715801239014, 'learning_rate': 9.583333333333335e-06, 'epoch': 2.08}


...

  0%|          | 0/60 [00:00<?, ?it/s]

{'eval_loss': 0.3784990906715393, 'eval_f1': 0.9189366590088612, 'eval_runtime': 6.4108, 'eval_samples_per_second': 148.498, 'eval_steps_per_second': 9.359, 'epoch': 49.0}
{'loss': 0.0004, 'grad_norm': 0.2580961287021637, 'learning_rate': 0.0, 'epoch': 50.0}


  0%|          | 0/60 [00:00<?, ?it/s]

{'eval_loss': 0.3775757849216461, 'eval_f1': 0.9207497533706018, 'eval_runtime': 6.3821, 'eval_samples_per_second': 149.167, 'eval_steps_per_second': 9.401, 'epoch': 50.0}
{'train_runtime': 11509.2287, 'train_samples_per_second': 33.338, 'train_steps_per_second': 2.085, 'train_loss': 0.03995643497010072, 'epoch': 50.0}


TrainOutput(global_step=24000, training_loss=0.03995643497010072, metrics={'train_runtime': 11509.2287, 'train_samples_per_second': 33.338, 'train_steps_per_second': 2.085, 'total_flos': 5.08287835559016e+16, 'train_loss': 0.03995643497010072, 'epoch': 50.0})

In [8]:
y_pred = trainer.predict(tokenized_dataset["test"])
y_pred.metrics

  0%|          | 0/61 [00:00<?, ?it/s]

{'test_loss': 0.16220751404762268,
 'test_f1': 0.9070753179002283,
 'test_runtime': 5.7191,
 'test_samples_per_second': 168.909,
 'test_steps_per_second': 10.666}

In [ ]:
trainer.save_model("finetunedTransformer_1")

: 